In [1]:
#第5章準備 実行は一度で良い(実行済み)

import zipfile

# zipファイルに含まれているファイルの取り出し
with zipfile.ZipFile("./data/ai.ja.zip") as myzip:
    myzip.extractall()

!cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt 


In [2]:
#40. 係り受け解析結果の読み込み(形態素)

class Morph:  # 形態素を表すクラスMorph
    def __init__(self, morph):
        surface, attribute = morph.split("\t")  # 各行は"表層形\t属性"となっているので,それをsplitする
        attribute = attribute.split(",")  # 属性はコンマ区切りになっているので,それをsplitする
        
        self.surface = surface
        self.base = attribute[6]
        self.pos = attribute[0]
        self.pos1 = attribute[1]  # それぞれ代入
        
File = "./data/ai.ja.txt.parsed"

sentences = []
morphs = []

with open(File, mode="r") as f:
    for line in f:
        if line[0] == "*":  # lineが"*"で始まっている行は係り受けを表す行なので、ここではcontinue
            continue
        elif line != "EOS\n":  # "EOS\n"は文の終わりを表すので、そうでない時にmorphsに形態素を表すMorphをappendしていく
            morphs.append(Morph(line))
        else:  # "EOS\n"は文の終わりを表すので、その時はsentencesに1文となっているmorphsをappendし、morphsを初期化
            sentences.append(morphs)
            morphs = []

for i in sentences[4]:
    print(vars(i))　　

# "vars()"はモジュール、クラス、インスタンス、あるいはそれ以外の __dict__ 属性を持つオブジェクトの、 __dict__ 属性を返す。

{'surface': '『', 'base': '『', 'pos': '記号', 'pos1': '括弧開'}
{'surface': '日本', 'base': '日本', 'pos': '名詞', 'pos1': '固有名詞'}
{'surface': '大', 'base': '大', 'pos': '接頭詞', 'pos1': '名詞接続'}
{'surface': '百科全書', 'base': '百科全書', 'pos': '名詞', 'pos1': '一般'}
{'surface': '(', 'base': '*\n', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'ニッポニカ', 'base': '*\n', 'pos': '名詞', 'pos1': '一般'}
{'surface': ')』', 'base': '*\n', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'の', 'base': 'の', 'pos': '助詞', 'pos1': '連体化'}
{'surface': '解説', 'base': '解説', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '情報', 'base': '情報', 'pos': '名詞', 'pos1': '一般'}
{'surface': '工学', 'base': '工学', 'pos': '名詞', 'pos1': '一般'}
{'surface': '者', 'base': '者', 'pos': '名詞', 'pos1': '接尾'}
{'surface': '・', 'base': '・', 'pos': '記号', 'pos1': '一般'}
{'surface': '通信', 'base': '通信', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': '工学', 'base': '工学', 'pos': '名詞', 

In [42]:
#41. 係り受け解析結果の読み込み(文節・係り受け)
import re
class Morph:
    def __init__(self, line):
        surface, attribute = line.split("\t")
        attribute = attribute.split(",")
        
        self.surface = surface
        self.base = attribute[6]
        self.pos = attribute[0]
        self.pos1 = attribute[1]

class Chunk:  # 文節を表すクラスChunk
    def __init__(self, morphs, dst, chunknum):  # 形態素列morphsと係り受け先dstと文節番号chunknumを受け取りselfに代入,そして係り元列srcsを初期化 
        self.morphs = morphs
        self.dst = dst
        self.srcs = []
        self.chunknum = chunknum
        
    def __str__(self):
    # "__str__"はオブジェクトの非公式の、あるいは表示に適した文字列表現を計算するために、
    # str(object)と組み込み関数format(),print()によって呼ばれる。戻り値はstringオブジェクト。
        return ''.join( mor.surface for mor in morphs )
        
class Sentence:  # 文を表すクラスSentence
    def __init__(self, chunks):  # 文節を表すchunksを受け取りselfに代入、
        self.chunks = chunks
        for i, chunk in enumerate(self.chunks):  # chunks内のインデックスをi、値(?)をchunkとしてfor文を回す
            if chunk.dst != -1:  #chunkの係先が-1でなかった場合
                self.chunks[chunk.dst].srcs.append(i)  # chunks内でchunkの係り先となっているとなっているところ(chunks[chunk.dst])の係り元列にiをappendする
        
File = "./data/ai.ja.txt.parsed"


morphs = []
chunks = []
sentences = []

# morphsの終わり=1つのchunk → chunksにappend
# chunksの終わり=1つのsentence → sentencesにappend
# sentencesの終わり=text全体
with open(File, mode="r") as f:
    for line in f:
        if line[0] == "*":  # "*"で始まっている行は文節の始まりを表すので、morphsに要素があるのなら、そこまでの結果をchunksにappendし、morphsを初期化
            if len(morphs) > 0:
                chunks.append(Chunk(morphs, dst, chunknum))  # Chunk()に1つの文節とその係り先を入れ、chunksにappendする
                #print( str(Chunk(morphs, dst)) )
                morphs = []
            dst = int(line.split(" ")[2].rstrip("D"))  # 文節の係り先dstに、その値を代入
            chunknum = int(line.split(" ")[1])
        elif line != "EOS\n":  # lineが"EOS\n"のでない時は、形態素列morphsにMorph(line)をappendする
            morphs.append(Morph(line))
        else:  # lineが"EOS\n"の時はそれは1文の終わりを表すので、chunksに最後のChunk(morphs, dst)をappendしてからsentencesにSentence(chunks)をappendし、morphsとchunksとdstを初期化
            chunks.append(Chunk(morphs, dst, chunknum))
            sentences.append(Sentence(chunks))
            morphs = []
            chunks = []
            dst = 0

#  sentencesにはtextの全てのSentenceがappendされているので、以下のように何文目かを選択して出力する。
for chunk in sentences[2].chunks:
    print([morph.surface for morph in chunk.morphs], chunk.chunknum, chunk.dst, chunk.srcs, )

for chunk in sentences[2].chunks:
    print([re.sub(r"[、。（）()「」『』]", "", "".join([morph.surface for morph in chunk.morphs]))], chunk.chunknum, chunk.dst, chunk.srcs)

['人工', '知能'] 0 17 []
['（', 'じん', 'こうち', 'のう', '、', '、'] 1 17 []
['AI'] 2 3 []
['〈', 'エーアイ', '〉', '）', 'と', 'は', '、'] 3 17 [2]
['「', '『', '計算'] 4 5 []
['（', '）', '』', 'という'] 5 9 [4]
['概念', 'と'] 6 9 []
['『', 'コンピュータ'] 7 8 []
['（', '）', '』', 'という'] 8 9 [7]
['道具', 'を'] 9 10 [5, 6, 8]
['用い', 'て'] 10 12 [9]
['『', '知能', '』', 'を'] 11 12 []
['研究', 'する'] 12 13 [10, 11]
['計算', '機', '科学'] 13 14 [12]
['（', '）', 'の'] 14 15 [13]
['一', '分野', '」', 'を'] 15 16 [14]
['指す'] 16 17 [15]
['語', '。'] 17 34 [0, 1, 3, 16]
['「', '言語', 'の'] 18 20 []
['理解', 'や'] 19 20 []
['推論', '、'] 20 21 [18, 19]
['問題', '解決', 'など', 'の'] 21 22 [20]
['知的', '行動', 'を'] 22 24 [21]
['人間', 'に'] 23 24 []
['代わっ', 'て'] 24 26 [22, 23]
['コンピューター', 'に'] 25 26 []
['行わ', 'せる'] 26 27 [24, 25]
['技術', '」', '、', 'または', '、'] 27 34 [26]
['「', '計算', '機'] 28 29 []
['（', 'コンピュータ', '）', 'による'] 29 31 [28]
['知的', 'な'] 30 31 []
['情報処理', 'システム', 'の'] 31 33 [29, 30]
['設計', 'や'] 32 33 []
['実現', 'に関する'] 33 34 [31, 32]
['研究', '分野', '」', 'と', 'も'] 34 35 [17, 27, 33

In [32]:
#42. 係り元と係り先の文節の表示
##↓このコードでは同じ文字列が出てきた時に対応できない。
##今回の問題では"『計算（）』という概念と『コンピュータ（）』という道具"について、どちらの"という"も"概念"という単語に係ってしまうことになる
##やり直し

import pprint

ChunkDict = {}

print(len(sentences[2].chunks))

ThisChunks = sentences[2].chunks
for chunk in ThisChunks:
    if chunk.dst == -1:
        continue
    elif "".join([morph.surface for morph in ThisChunks[chunk.dst].morphs]) not in ChunkDict:
        ChunkDict.setdefault("".join([morph.surface for morph in ThisChunks[chunk.dst].morphs]), ["".join([morph.surface for morph in chunk.morphs])])
    else:
        ChunkDict["".join([morph.surface for morph in ThisChunks[chunk.dst].morphs])].append("".join([morph.surface for morph in chunk.morphs]))

pprint.pprint(ChunkDict, sort_dicts=False)

36
{'語。': ['人工知能', '（じんこうちのう、、', '〈エーアイ〉）とは、', '指す'],
 '〈エーアイ〉）とは、': ['AI'],
 '（）』という': ['「『計算', '『コンピュータ'],
 '道具を': ['（）』という', '概念と', '（）』という'],
 '用いて': ['道具を'],
 '研究する': ['用いて', '『知能』を'],
 '計算機科学': ['研究する'],
 '（）の': ['計算機科学'],
 '一分野」を': ['（）の'],
 '指す': ['一分野」を'],
 '研究分野」とも': ['語。', '技術」、または、', '実現に関する'],
 '推論、': ['「言語の', '理解や'],
 '問題解決などの': ['推論、'],
 '知的行動を': ['問題解決などの'],
 '代わって': ['知的行動を', '人間に'],
 '行わせる': ['代わって', 'コンピューターに'],
 '技術」、または、': ['行わせる'],
 '（コンピュータ）による': ['「計算機'],
 '情報処理システムの': ['（コンピュータ）による', '知的な'],
 '実現に関する': ['情報処理システムの', '設計や'],
 'される。': ['研究分野」とも']}


In [82]:
#42. 係り元と係り先の文節の表示
#辞書型でやろうとしてもうまくいかない

import pprint

ChunkDict = {}
DstList
print(len(sentences[2].chunks))

ThisChunks = sentences[2].chunks
for chunk in ThisChunks:
    if chunk.dst == -1:
        continue
    elif "".join([morph.surface for morph in ThisChunks[chunk.dst].morphs]) not in ChunkDict:
        ChunkDict.setdefault("".join([morph.surface for morph in ThisChunks[chunk.dst].morphs]), ["".join([morph.surface for morph in chunk.morphs])])
    else:
        ChunkDict["".join([morph.surface for morph in ThisChunks[chunk.dst].morphs])].append("".join([morph.surface for morph in chunk.morphs]))

pprint.pprint(ChunkDict, sort_dicts=False)

36
{'語。': ['人工知能', '（じんこうちのう、、', '〈エーアイ〉）とは、', '指す'],
 '〈エーアイ〉）とは、': ['AI'],
 '（）』という': ['「『計算', '『コンピュータ'],
 '道具を': ['（）』という', '概念と', '（）』という'],
 '用いて': ['道具を'],
 '研究する': ['用いて', '『知能』を'],
 '計算機科学': ['研究する'],
 '（）の': ['計算機科学'],
 '一分野」を': ['（）の'],
 '指す': ['一分野」を'],
 '研究分野」とも': ['語。', '技術」、または、', '実現に関する'],
 '推論、': ['「言語の', '理解や'],
 '問題解決などの': ['推論、'],
 '知的行動を': ['問題解決などの'],
 '代わって': ['知的行動を', '人間に'],
 '行わせる': ['代わって', 'コンピューターに'],
 '技術」、または、': ['行わせる'],
 '（コンピュータ）による': ['「計算機'],
 '情報処理システムの': ['（コンピュータ）による', '知的な'],
 '実現に関する': ['情報処理システムの', '設計や'],
 'される。': ['研究分野」とも']}


In [84]:
#42. 係り元と係り先の文節の表示

sentence = sentences[2]
for chunk in sentence.chunks:
    if int(chunk.dst) != -1:
        SrcChunk = ''.join([morph.surface if morph.pos != '記号' else '' for morph in chunk.morphs])  # morphが記号でなければmorph.surfaceを取る
        DstChunk = ''.join([morph.surface if morph.pos != '記号' else '' for morph in sentence.chunks[int(chunk.dst)].morphs])  # morphが記号でなければ係り先のmorph.surfaceを取る
        print(SrcChunk, "\t", DstChunk)

人工知能 	 語
じんこうちのう 	 語
AI 	 エーアイとは
エーアイとは 	 語
計算 	 という
という 	 道具を
概念と 	 道具を
コンピュータ 	 という
という 	 道具を
道具を 	 用いて
用いて 	 研究する
知能を 	 研究する
研究する 	 計算機科学
計算機科学 	 の
の 	 一分野を
一分野を 	 指す
指す 	 語
語 	 研究分野とも
言語の 	 推論
理解や 	 推論
推論 	 問題解決などの
問題解決などの 	 知的行動を
知的行動を 	 代わって
人間に 	 代わって
代わって 	 行わせる
コンピューターに 	 行わせる
行わせる 	 技術または
技術または 	 研究分野とも
計算機 	 コンピュータによる
コンピュータによる 	 情報処理システムの
知的な 	 情報処理システムの
情報処理システムの 	 実現に関する
設計や 	 実現に関する
実現に関する 	 研究分野とも
研究分野とも 	 される


In [91]:
#43. 名詞を含む文節が動詞を含む文節に係るものを抽出

sentence = sentences[2]
SrcFlag = False
DstFlag = False

for chunk in sentence.chunks:
    if int(chunk.dst) != -1:
        SrcFlag = True in [True if morph.pos == '名詞' else False for morph in chunk.morphs]  # 文節の中に名詞があったらフラグがオンに
        DstFlag = True in [True if morph.pos == '動詞' else False for morph in sentence.chunks[int(chunk.dst)].morphs]  # 文節の中に動詞があったらフラグがオンに
        
        SrcChunk = ''.join([morph.surface if morph.pos != '記号' else '' for morph in chunk.morphs])  # morphが記号でなければmorph.surfaceを取る
        DstChunk = ''.join([morph.surface if morph.pos != '記号' else '' for morph in sentence.chunks[int(chunk.dst)].morphs])  # morphが記号でなければ係り先のmorph.surfaceを取る
        
        if SrcFlag and DstFlag:
            #print(SrcFlag)
            #print(DstFlag)
            print(SrcChunk, "\t", DstChunk)
        
        SrcFlag = False
        DstFlag = False

道具を 	 用いて
知能を 	 研究する
一分野を 	 指す
知的行動を 	 代わって
人間に 	 代わって
コンピューターに 	 行わせる
研究分野とも 	 される


In [ ]:
#44. 係り受け木の可視化

